In [1]:
from numpy import dtype
from kdg import kdf, kdn, kdcnn
from kdg.utils import get_ece, plot_reliability
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import activations
from tensorflow.keras import backend as bknd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os 
import numpy as np
import openml
from sklearn.metrics import cohen_kappa_score
from kdg.utils import get_ece
from numpy import min_scalar_type
from sklearn.mixture import GaussianMixture
from sklearn.utils.validation import check_array, check_is_fitted, check_X_y
from sklearn.ensemble import RandomForestClassifier as rf 
import numpy as np
from scipy.stats import multivariate_normal
import warnings
from sklearn.covariance import MinCovDet, fast_mcd, GraphicalLassoCV, LedoitWolf, EmpiricalCovariance, OAS, EllipticEnvelope, log_likelihood
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split


In [2]:
dataset = openml.datasets.get_dataset(1050)
X, y, is_categorical, _ = dataset.get_data(
                dataset_format="array", target=dataset.default_target_attribute
            )

#feature = [475, 433]
#X = X[:,feature]
#X /= np.max(np.linalg.norm(X,2,axis=1))
total_sample = X.shape[0]
train_samples = np.logspace(
            np.log10(100),
            np.log10(total_sample-1000),
            num=5,
            endpoint=True,
            dtype=int
        )
X_train, X_test, y_train, y_test = train_test_split(
                     X, y, test_size=.33, random_state=44)

min_val = np.min(X_train,axis=0)
max_val = np.max(X_train, axis=0)
    
X_train = (X_train-min_val)/(max_val-min_val+1e-12)
X_test = (X_test-min_val)/(max_val-min_val+1e-12)


In [3]:
compile_kwargs = {
        "loss": "binary_crossentropy",
        "optimizer": keras.optimizers.Adam(3e-4),
    }
callback = keras.callbacks.EarlyStopping(monitor="loss", patience=10, verbose=True)
fit_kwargs = {
        "epochs": 2000,
        "batch_size": 32,
        "verbose": False,
        "callbacks": [callback],
    }

In [4]:
def getNN(input_size, num_classes, layer_size=2000):
    network_base = keras.Sequential()
    initializer = keras.initializers.random_normal(seed=0)
    network_base.add(keras.layers.Dense(layer_size, kernel_initializer=initializer, input_shape=(input_size,)))
    network_base.add(keras.layers.Activation(activations.relu))
    network_base.add(keras.layers.Dense(layer_size, kernel_initializer=initializer))
    network_base.add(keras.layers.Activation(activations.relu))
    network_base.add(keras.layers.Dense(layer_size, kernel_initializer=initializer))
    network_base.add(keras.layers.Activation(activations.relu))
    network_base.add(keras.layers.Dense(layer_size, kernel_initializer=initializer))
    network_base.add(keras.layers.Activation(activations.relu))
    network_base.add(keras.layers.Dense(units=num_classes, activation="softmax", kernel_initializer=initializer))
    network_base.compile(**compile_kwargs)
    return network_base

In [5]:
nn = getNN(input_size=X.shape[1], num_classes=len(np.unique(y)), layer_size=1000)
history = nn.fit(X_train, keras.utils.to_categorical(y_train), **fit_kwargs)


Metal device set to: Apple M1 Max


2023-08-04 09:47:43.058632: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-04 09:47:43.059217: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-08-04 09:47:43.270842: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-04 09:47:43.401324: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 71: early stopping


In [6]:
model_kdn = kdn(network=nn)
model_kdn.fit(X_train, y_train, epsilon=1e-6, mul=20)

In [7]:
proba_kdn = model_kdn.predict_proba(X_test, distance='Geodesic')
proba_dn = model_kdn.network.predict(X_test)
predicted_label_kdn = np.argmax(proba_kdn, axis = 1)
predicted_label_dn = np.argmax(proba_dn, axis = 1)

[221 152 841 738 740 752 829 193 538 514 230 394 391 333 320 887 224 738
 368  45 606 192 620 621 621 965 124  42 466 624 224 752  18 818 452 381
  67 716 972 842 517  91 941 186 913 949 751 366 195 350 464 827 947 904
 633 240 183 649 334 707 162 304 743 245 608  52 488 949  96 253  98 248
 143 521 845 477 617  81 196 143 585 157 608 525 293 824  53 327 859 523
 224 482 622 323 273 187 436 193 872  63 665 651 460 191 645 950 579 540
 502 680 728 245 560 874 354 327 429 504 352 659 941 476   9 364 544 304
  37 638  60 511 445 896 735 820 447 133 953 167 272 625 155 627   7 186
 481 298 520 186 515  57 575  99 544 556 917 636 912  34 706 616 284 402
 864 488 794 636 224 186 836 453 916 322 880 341 944 129 736  88 175 261
 784 561 725 565 453 658 692 709 571 379   7 542 619 491 399 646 297   5
 963 452 403 717 383 784 765 499 333 133 608   3 137 225 128 174 831 509
 908   7  43 625 502 216 881  86 711 744   7 508 637 438 112 731 127 794
 460 254 385 161 446 416 895 252 694 450 331 946 49

2023-08-04 09:48:06.623747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [9]:
np.mean(predicted_label_dn==y_test)

0.8934108527131783

In [10]:
np.mean(predicted_label_kdn==y_test)

0.8934108527131783